In [1]:
import psycopg2
from PIL import Image
import requests
from io import BytesIO

db = psycopg2.connect("dbname=africa user=etienne host=/var/run/postgresql/")
cur = db.cursor()

limit = 10

url = """https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/-2.547104,10.01967,17/300x300?access_token=pk.eyJ1IjoiZXRpYmR2IiwiYSI6ImNpejhvdWJmcjAwMW8yd28weTkzMnA1aDkifQ.i8UKq0M_sIN1qq8F6UAgFw"""

response = requests.get(url)
img = Image.open(BytesIO(response.content))

In [6]:
from matplotlib.pyplot import imshow
import numpy as np
%matplotlib inline

print(np.shape(np.asarray(img)))

(300, 300, 3)
